In [1]:
import cdsapi

In [5]:
# # Example taken from:
# # https://cds-beta.climate.copernicus.eu/how-to-api#use-the-cds-api-client-for-data-access

# client = cdsapi.Client()

# dataset = 'reanalysis-era5-pressure-levels'
# request = {
#     'product_type': ['reanalysis'],
#     'variable': ['geopotential'],
#     'year': ['2024'],
#     'month': ['03'],
#     'day': ['01'],
#     'time': ['13:00'],
#     'pressure_level': ['1000'],
#     'data_format': 'grib',
# }
# target = 'download.grib'

# client.retrieve(dataset, request, target)

2024-09-06 23:15:05,807 INFO Request ID is 218af5b3-1fee-46ae-9ec8-ee046ce8bf12
2024-09-06 23:15:05,882 INFO status has been updated to accepted
2024-09-06 23:15:54,761 INFO status has been updated to successful


2e519a585f4194cb8017839b4fce8703.grib:   0%|          | 0.00/1.98M [00:00<?, ?B/s]

'download.grib'

In [7]:
dataset = "sis-ecde-climate-indicators"
request = {
    'variable': ['hot_days', 'frost_days', 'duration_of_meteorological_droughts'],
    'origin': 'reanalysis',
    'temporal_aggregation': ['yearly'],
    'spatial_aggregation': 'gridded',
    'other_parameters': ['30_c']
}

client = cdsapi.Client()
client.retrieve(dataset, request).download()

2024-09-14 14:05:26,029 INFO Request ID is c77b0967-a993-4e2a-8ef2-df95a25057dc
2024-09-14 14:05:26,092 INFO status has been updated to accepted
2024-09-14 14:05:27,677 INFO status has been updated to running
2024-09-14 14:05:38,559 INFO Creating download object as zip with files:
['06_hot_days-reanalysis-yearly-30deg-grid-1940-2023-v1.0.nc', '11_frost_days-reanalysis-yearly-grid-1940-2023-v1.0.nc', '19_duration_of_meteorological_droughts-reanalysis-yearly-grid-1940-2023-v1.0.nc']
2024-09-14 14:05:38,561 INFO status has been updated to successful


'47185a9390b89ceb75159c8bdce5a232.zip'

In [8]:
import zipfile

In [13]:
zip_name = '47185a9390b89ceb75159c8bdce5a232'

path_to_zip_file = f'{zip_name}.zip'
directory_to_extract_to = f'data/{zip_name}'

with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [14]:
import netCDF4

In [22]:
def read_nc_file(file_name):
    path_to_file = f'data/{zip_name}/{file_name}'
    file2read = netCDF4.Dataset(path_to_file,'r')
    #file2read = netcdf.NetCDFFile(path,'r')
    keys = file2read.variables.keys()
    print(keys)
    print(file2read)
    data = file2read.variables['time'][:]
    print(data)
    file2read.close()

In [24]:
for name in ['01_mean_temperature-reanalysis-yearly-grid-1940-2023-v1.0.nc',
             '06_hot_days-reanalysis-yearly-30deg-grid-1940-2023-v1.0.nc',
             '11_frost_days-reanalysis-yearly-grid-1940-2023-v1.0.nc',
             '19_duration_of_meteorological_droughts-reanalysis-yearly-grid-1940-2023-v1.0.nc']:
    read_nc_file(name)
    print('\n\n\n')

dict_keys(['time', 'realization', 'lat', 'lon', 't2m'])
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.7
    institution: European Centre for Medium-Range Weather Forecasts
    history: 2024-04-02T13:52 GRIB to CDM+CF via cfgrib-0.9.9.1/ecCodes-2.27.0 with {"source": "/nfs/compute-0014/data3/adaptor.mars.internal-1712065708.8353992-6424-6-39247118-b7b6-4b87-8957-46d0ba425a8f.grib", "filter_by_keys": {}, "encode_cf": ["parameter", "time", "geography", "vertical"]}
    source: ECMWF
    dimensions(sizes): time(84), lat(185), lon(271)
    variables(dimensions): int64 time(time), int64 realization(), float64 lat(lat), float64 lon(lon), float32 t2m(time, lat, lon)
    groups: 
[    0   366   731  1096  1461  1827  2192  2557  2922  3288  3653  4018
  4383  4749  5114  5479  5844  6210  6575  6940  7305  7671  8036  8401
  8766  9132  9497  9862 10227 10593 10958 11323 11688 12054 12419 12784
 13149 13515 13880 14245 14610 14976 15

In [25]:
file_name = '01_mean_temperature-reanalysis-yearly-grid-1940-2023-v1.0.nc'
path_to_file = f'data/{zip_name}/{file_name}'

file2read = netCDF4.Dataset(path_to_file,'r')

time = file2read.variables['time'][:]
lat = file2read.variables['lat'][:]
lon = file2read.variables['lon'][:]
mean_temperature = file2read.variables['t2m'][:]

file2read.close()

In [29]:
file_name = '06_hot_days-reanalysis-yearly-30deg-grid-1940-2023-v1.0.nc'

path_to_file = f'data/{zip_name}/{file_name}'
file2read = netCDF4.Dataset(path_to_file,'r')
hot_days = file2read.variables['t2m'][:]

file2read.close()

In [30]:
file_name = '11_frost_days-reanalysis-yearly-grid-1940-2023-v1.0.nc'

path_to_file = f'data/{zip_name}/{file_name}'
file2read = netCDF4.Dataset(path_to_file,'r')
frost_days = file2read.variables['t2m'][:]

file2read.close()

In [31]:
file_name = '19_duration_of_meteorological_droughts-reanalysis-yearly-grid-1940-2023-v1.0.nc'

path_to_file = f'data/{zip_name}/{file_name}'
file2read = netCDF4.Dataset(path_to_file,'r')
droughts = file2read.variables['dmd'][:]

file2read.close()

In [32]:
def iter_3D(matrix):
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            for k in range(matrix.shape[2]):
                yield i, j, k

In [34]:
l = []

l.append('time, lat, lon, mean_t, hot_d, cold_d, droughts')

for i, j, k in iter_3D(mean_temperature):
    l.append('%d,%.2f,%.2f,%.2f,%d,%d,%d' %((time[i]/365 + 1940), lat[j], lon[k],
                                    mean_temperature[i, j, k]-273.15,
                                    hot_days[i, j, k],
                                    frost_days[i, j, k],
                                    droughts[i, j, k]))

with open('file.csv', 'w') as f:
    f.write("\n".join(l))